In [33]:
#Imports
import os
import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from google.colab import drive

In [2]:
#Mount and access drive
drive.mount('/content/drive',force_remount=True)
os.chdir('/content/drive/My Drive/Few-Shot-Tamil')
print("Change successful.")

Mounted at /content/drive
Change successful.


In [3]:
#Load the datasets
#One shot
one_shot_real = pd.read_csv('one-shot-real.csv',  sep='\t')
one_shot_real.head()

,English version,Label,News,Author,Date,URL
0,Actor Jayam Ravi has Corona,['entertainment'],நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...
1,A football god about the 'hands of God',['misc'],‘கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,ஆதி,01-Dec-20,https://www.hindutamil.in/news/supplements/ila...
2,Cameras mandatory in school buses: Tamil Nadu ...,['politics'],பள்ளி பேருந்துகளில் கேமரா கட்டாயம்: தமிழக அரச...,செய்திப்பிரிவு,21-Oct-22,https://www.hindutamil.in/news/tamilnadu/88576...
3,36th National Games | 4 gold medals for Tamil ...,['sport'],36-வது தேசிய விளையாட்டு | ஒரே நாளில் தமிழகத்த...,செய்திப்பிரிவு,02-Oct-22,https://www.hindutamil.in/news/sports/877427-3...
4,Introducing NoiseFit Core 2 Smartwatch: Amazin...,['tech'],நாய்ஸ்ஃபிட் கோர் 2 ஸ்மார்ட்வாட்ச் அறிமுகம்: 5...,செய்திப்பிரிவு,22-Aug-22,https://www.hindutamil.in/news/technology/8467...


In [4]:
#Load the datasets
#One shot
one_shot_fake = pd.read_csv('one-shot-fake.csv',  sep='\t')
one_shot_fake.head()

,English version,Label,News,Author,Date,URL
0,Says Daniel Radcliffe has the coronavirus.,entertainment,டேனியல் ராட்கிளிஃப் கொரோனா வைரஸால் பாதிக்கப்பட...,Tweets,"stated on March 10, 2020 in in a tweet:",/factchecks/2020/mar/11/tweets/no-daniel-radcl...
1,"Says ""there is no"" COVID-19 virus.",miscellaneous,"COVID-19 வைரஸ் ""இல்லை"" என்கிறார்.",Facebook posts,"stated on March 19, 2020 in a post:",/factchecks/2020/mar/19/facebook-posts/faceboo...
2,"Says President Donald Trump said, ""People are ...",Politics,"அமெரிக்க அதிபர் டொனால்ட் டிரம்ப், “இதுவரை இறக்...",Facebook posts,"stated on March 18, 2020 in a Facebook post:",/factchecks/2020/mar/19/facebook-posts/no-trum...
3,Kieron Pollard is good: Viral 'Kieron Pollard ...,sport,Kieron Pollard நல்லாதான் இருக்காரு: Viral ஆன ‘...,undefined,"Jan 29, 2021, 07:56 PM IST",https://zeenews.india.com/tamil/social/kieron-...
4,PIB fact check: Link between 5G radiation and ...,tech,PIB fact check: 5 ஜி கதிர்வீச்சுக்கும் COVID-1...,undefined,"May 8, 2021, 11:20 PM IST",https://zeenews.india.com/tamil/social/pib-fac...


In [5]:
#Load the datasets
#Three shot
three_shot_real = pd.read_csv('three-shot-real.csv',  sep='\t')
three_shot_real.describe()

,English version,Label,News,Author,Date,URL
count,15,15,15,15,15,15
unique,15,5,15,6,14,15
top,Actor Jayam Ravi has Corona,['entertainment'],நடிகர் ஜெயம் ரவிக்கு,செய்திப்பிரிவு,20-Oct-22,https://www.dailythanthi.com/Cinema/CinemaNews...
freq,1,3,1,8,2,1


In [6]:
#Load the datasets
#Three shot
three_shot_fake = pd.read_csv('three-shot-fake.csv',  sep='\t')
three_shot_fake.describe()

,English version,Label,News,Author,Date,URL
count,15,15,15,15,15,15
unique,15,9,15,8,15,15
top,Says Daniel Radcliffe has the coronavirus.,miscellaneous,டேனியல் ராட்கிளிஃப் கொரோனா வைரஸால் பாதிக்கப்பட...,Bloggers,"stated on March 10, 2020 in in a tweet:",/factchecks/2020/mar/11/tweets/no-daniel-radcl...
freq,1,3,1,3,1,1


In [7]:
#Load the datasets
#Five shot
five_shot_real = pd.read_csv('five-shot-real.csv',  sep='\t')
five_shot_real.describe()

,English version,Label,News,Author,Date,URL
count,25,25,25,25,25,25
unique,25,5,25,7,22,25
top,Actor Jayam Ravi has Corona,['entertainment'],நடிகர் ஜெயம் ரவிக்கு,செய்திப்பிரிவு,20-Oct-22,https://www.dailythanthi.com/Cinema/CinemaNews...
freq,1,5,1,14,4,1


In [8]:
#Load the datasets
#Five shot
five_shot_fake = pd.read_csv('five-shot-fake.csv',  sep='\t')
five_shot_fake.describe()

,English version,Label,News,Author,Date,URL
count,25,25,25,25,25,25
unique,25,11,25,17,25,25
top,Says Daniel Radcliffe has the coronavirus.,['politics'],டேனியல் ராட்கிளிஃப் கொரோனா வைரஸால் பாதிக்கப்பட...,Facebook posts,"stated on March 10, 2020 in in a tweet:",/factchecks/2020/mar/11/tweets/no-daniel-radcl...
freq,1,4,1,3,1,1


In [9]:
#Fix the labels for the fake news mini-corpora
#Fix one shot fake
one_shot_fake['Label'] = one_shot_fake['Label'].apply(lambda x: re.sub('[^a-zA-Z]+', '', str(x)))
one_shot_fake['Label'].unique()

array(['entertainment', 'miscellaneous', 'Politics', 'sport', 'tech'],
      dtype=object)

In [10]:
#Fix the labels for the fake news mini-corpora
#Fix three shot fake
three_shot_fake['Label'] = three_shot_fake['Label'].apply(lambda x: re.sub('[^a-zA-Z]+', '', str(x)))
three_shot_fake['Label'].unique()

array(['entertainment', 'miscellaneous', 'politics', 'sport', 'tech'],
      dtype=object)

In [11]:
#Fix the labels for the fake news mini-corpora
#Fix three shot fake
three_shot_fake['Label'] = three_shot_fake['Label'].apply(lambda x: re.sub('[^a-zA-Z]+', '', str(x)))
three_shot_fake['Label'].unique()

array(['entertainment', 'miscellaneous', 'politics', 'sport', 'tech'],
      dtype=object)

In [12]:
#Fix the labels for the fake news mini-corpora
#Fix five shot fake
five_shot_fake['Label'] = five_shot_fake['Label'].apply(lambda x: re.sub('[^a-zA-Z]+', '', str(x)))
five_shot_fake['Label'].unique()

array(['entertainment', 'miscellaneous', 'politics', 'sport', 'tech'],
      dtype=object)

In [13]:
def tokenize(data):
  generated_token = list(data.split())
  return generated_token

In [14]:
ps = nltk.PorterStemmer()
def stemming(list_of_words):
  stemmed_list = [ps.stem(word) for word in list_of_words]
  return stemmed_list

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
#English translation preprocessing
def eng_preprocessing(dataframe):
  #Remove non-alphanumeric symbols
  dataframe['only_alphanumeric'] = dataframe['English version'].apply(lambda x: re.sub(r'[^a-zA-Z ]+', '', x.lower()))

  #Remove stop-words
  dataframe['imp_text'] = dataframe['only_alphanumeric'].apply(lambda x : ' '.join([word for word in x.split() if not word in set(stopwords.words('english'))]))

  #Tokenize the sentences
  dataframe['tokenized_text'] = dataframe['imp_text'].apply(lambda x: tokenize(x))

  #Apply stemming
  dataframe['stemmed_text'] = dataframe['tokenized_text'].apply(lambda x: stemming(x))

In [17]:
eng_preprocessing(one_shot_real)
one_shot_real.head()

,English version,Label,News,Author,Date,URL,only_alphanumeric,imp_text,tokenized_text,stemmed_text
0,Actor Jayam Ravi has Corona,['entertainment'],நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...,actor jayam ravi has corona,actor jayam ravi corona,"[actor, jayam, ravi, corona]","[actor, jayam, ravi, corona]"
1,A football god about the 'hands of God',['misc'],‘கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,ஆதி,01-Dec-20,https://www.hindutamil.in/news/supplements/ila...,a football god about the hands of god,football god hands god,"[football, god, hands, god]","[footbal, god, hand, god]"
2,Cameras mandatory in school buses: Tamil Nadu ...,['politics'],பள்ளி பேருந்துகளில் கேமரா கட்டாயம்: தமிழக அரச...,செய்திப்பிரிவு,21-Oct-22,https://www.hindutamil.in/news/tamilnadu/88576...,cameras mandatory in school buses tamil nadu g...,cameras mandatory school buses tamil nadu gove...,"[cameras, mandatory, school, buses, tamil, nad...","[camera, mandatori, school, buse, tamil, nadu,..."
3,36th National Games | 4 gold medals for Tamil ...,['sport'],36-வது தேசிய விளையாட்டு | ஒரே நாளில் தமிழகத்த...,செய்திப்பிரிவு,02-Oct-22,https://www.hindutamil.in/news/sports/877427-3...,th national games gold medals for tamil nadu...,th national games gold medals tamil nadu one day,"[th, national, games, gold, medals, tamil, nad...","[th, nation, game, gold, medal, tamil, nadu, o..."
4,Introducing NoiseFit Core 2 Smartwatch: Amazin...,['tech'],நாய்ஸ்ஃபிட் கோர் 2 ஸ்மார்ட்வாட்ச் அறிமுகம்: 5...,செய்திப்பிரிவு,22-Aug-22,https://www.hindutamil.in/news/technology/8467...,introducing noisefit core smartwatch amazing ...,introducing noisefit core smartwatch amazing f...,"[introducing, noisefit, core, smartwatch, amaz...","[introduc, noisefit, core, smartwatch, amaz, f..."


In [18]:
eng_preprocessing(one_shot_fake)
one_shot_fake.head()

,English version,Label,News,Author,Date,URL,only_alphanumeric,imp_text,tokenized_text,stemmed_text
0,Says Daniel Radcliffe has the coronavirus.,entertainment,டேனியல் ராட்கிளிஃப் கொரோனா வைரஸால் பாதிக்கப்பட...,Tweets,"stated on March 10, 2020 in in a tweet:",/factchecks/2020/mar/11/tweets/no-daniel-radcl...,says daniel radcliffe has the coronavirus,says daniel radcliffe coronavirus,"[says, daniel, radcliffe, coronavirus]","[say, daniel, radcliff, coronaviru]"
1,"Says ""there is no"" COVID-19 virus.",miscellaneous,"COVID-19 வைரஸ் ""இல்லை"" என்கிறார்.",Facebook posts,"stated on March 19, 2020 in a post:",/factchecks/2020/mar/19/facebook-posts/faceboo...,says there is no covid virus,says covid virus,"[says, covid, virus]","[say, covid, viru]"
2,"Says President Donald Trump said, ""People are ...",Politics,"அமெரிக்க அதிபர் டொனால்ட் டிரம்ப், “இதுவரை இறக்...",Facebook posts,"stated on March 18, 2020 in a Facebook post:",/factchecks/2020/mar/19/facebook-posts/no-trum...,says president donald trump said people are dy...,says president donald trump said people dying ...,"[says, president, donald, trump, said, people,...","[say, presid, donald, trump, said, peopl, die,..."
3,Kieron Pollard is good: Viral 'Kieron Pollard ...,sport,Kieron Pollard நல்லாதான் இருக்காரு: Viral ஆன ‘...,undefined,"Jan 29, 2021, 07:56 PM IST",https://zeenews.india.com/tamil/social/kieron-...,kieron pollard is good viral kieron pollard de...,kieron pollard good viral kieron pollard death...,"[kieron, pollard, good, viral, kieron, pollard...","[kieron, pollard, good, viral, kieron, pollard..."
4,PIB fact check: Link between 5G radiation and ...,tech,PIB fact check: 5 ஜி கதிர்வீச்சுக்கும் COVID-1...,undefined,"May 8, 2021, 11:20 PM IST",https://zeenews.india.com/tamil/social/pib-fac...,pib fact check link between g radiation and se...,pib fact check link g radiation second wave co...,"[pib, fact, check, link, g, radiation, second,...","[pib, fact, check, link, g, radiat, second, wa..."


In [19]:
eng_preprocessing(three_shot_real)
three_shot_real.head()

,English version,Label,News,Author,Date,URL,only_alphanumeric,imp_text,tokenized_text,stemmed_text
0,Actor Jayam Ravi has Corona,['entertainment'],நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...,actor jayam ravi has corona,actor jayam ravi corona,"[actor, jayam, ravi, corona]","[actor, jayam, ravi, corona]"
1,Pooja Hegde regrets the film's failure,['entertainment'],படங்கள் தோல்வியால் வருந்தும் பூஜா ஹெக்டே,தினத்தந்தி,22-10-2022 02:50,https://www.dailythanthi.com/Cinema/CinemaNews...,pooja hegde regrets the films failure,pooja hegde regrets films failure,"[pooja, hegde, regrets, films, failure]","[pooja, hegd, regret, film, failur]"
2,Images for Diwali,['entertainment'],தீபாவளிக்கு வரும் படங்கள்,தினத்தந்தி,17-10-2022 10:20,https://www.dailythanthi.com/Cinema/CinemaNews...,images for diwali,images diwali,"[images, diwali]","[imag, diwali]"
3,A football god about the 'hands of God',['misc'],‘கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,ஆதி,01-Dec-20,https://www.hindutamil.in/news/supplements/ila...,a football god about the hands of god,football god hands god,"[football, god, hands, god]","[footbal, god, hand, god]"
4,200 km on two jugs of water. Car trip!,['misc'],இரண்டு குடம் தண்ணீரில் 200 கி.மீ. கார் பயணம்!,அண்டன் பிரகாஷ்,16-Nov-21,https://www.hindutamil.in/news/supplements/ila...,km on two jugs of water car trip,km two jugs water car trip,"[km, two, jugs, water, car, trip]","[km, two, jug, water, car, trip]"


In [20]:
eng_preprocessing(three_shot_fake)
three_shot_fake.head()

,English version,Label,News,Author,Date,URL,only_alphanumeric,imp_text,tokenized_text,stemmed_text
0,Says Daniel Radcliffe has the coronavirus.,entertainment,டேனியல் ராட்கிளிஃப் கொரோனா வைரஸால் பாதிக்கப்பட...,Tweets,"stated on March 10, 2020 in in a tweet:",/factchecks/2020/mar/11/tweets/no-daniel-radcl...,says daniel radcliffe has the coronavirus,says daniel radcliffe coronavirus,"[says, daniel, radcliffe, coronavirus]","[say, daniel, radcliff, coronaviru]"
1,"Says Kobe Bryant's wife ""recorded her suicide ...",entertainment,"கோபி பிரையன்ட்டின் மனைவி ""இன்று காலை மனச்சோர்வ...",Bloggers,"stated on February 3, 2020 in a blog post:",/factchecks/2020/feb/03/blog-posting/headline-...,says kobe bryants wife recorded her suicide af...,says kobe bryants wife recorded suicide depres...,"[says, kobe, bryants, wife, recorded, suicide,...","[say, kobe, bryant, wife, record, suicid, depr..."
2,Says Bill Cosby was framed for sexual assault ...,entertainment,பில் காஸ்பி என்பிசியை வாங்குவதைத் தடுப்பதற்காக...,Facebook posts,"stated on June 14, 2017 in a Facebook post:",/factchecks/2019/nov/04/facebook-posts/no-bill...,says bill cosby was framed for sexual assault ...,says bill cosby framed sexual assault illumina...,"[says, bill, cosby, framed, sexual, assault, i...","[say, bill, cosbi, frame, sexual, assault, ill..."
3,"Says ""there is no"" COVID-19 virus.",miscellaneous,"COVID-19 வைரஸ் ""இல்லை"" என்கிறார்.",Facebook posts,"stated on March 19, 2020 in a post:",/factchecks/2020/mar/19/facebook-posts/faceboo...,says there is no covid virus,says covid virus,"[says, covid, virus]","[say, covid, viru]"
4,Says a mall Santa beat up a child molester.,miscellaneous,ஒரு மால் சாண்டா ஒரு குழந்தை துஷ்பிரயோகம் செய்ப...,Bloggers,"stated on November 7, 2019 in a Facebook post:",/factchecks/2019/nov/26/blog-posting/dont-fall...,says a mall santa beat up a child molester,says mall santa beat child molester,"[says, mall, santa, beat, child, molester]","[say, mall, santa, beat, child, molest]"


In [21]:
eng_preprocessing(five_shot_real)
five_shot_real.head()

,English version,Label,News,Author,Date,URL,only_alphanumeric,imp_text,tokenized_text,stemmed_text
0,Actor Jayam Ravi has Corona,['entertainment'],நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...,actor jayam ravi has corona,actor jayam ravi corona,"[actor, jayam, ravi, corona]","[actor, jayam, ravi, corona]"
1,Pooja Hegde regrets the film's failure,['entertainment'],படங்கள் தோல்வியால் வருந்தும் பூஜா ஹெக்டே,தினத்தந்தி,22-10-2022 02:50,https://www.dailythanthi.com/Cinema/CinemaNews...,pooja hegde regrets the films failure,pooja hegde regrets films failure,"[pooja, hegde, regrets, films, failure]","[pooja, hegd, regret, film, failur]"
2,Images for Diwali,['entertainment'],தீபாவளிக்கு வரும் படங்கள்,தினத்தந்தி,17-10-2022 10:20,https://www.dailythanthi.com/Cinema/CinemaNews...,images for diwali,images diwali,"[images, diwali]","[imag, diwali]"
3,Simbu debuts in Bollywood,['entertainment'],பாலிவுட்டில் அறிமுகமாகிறார் சிம்பு,தினத்தந்தி,16-10-2022 03:49,https://www.dailythanthi.com/Cinema/CinemaNews...,simbu debuts in bollywood,simbu debuts bollywood,"[simbu, debuts, bollywood]","[simbu, debut, bollywood]"
4,3rd song release from 'Prince',['entertainment'],'பிரின்ஸ்' படத்தின் 3வது பாடல் வெளியீடு,தினத்தந்தி,14-10-2022 16:31,https://www.dailythanthi.com/Cinema/CinemaNews...,rd song release from prince,rd song release prince,"[rd, song, release, prince]","[rd, song, releas, princ]"


In [22]:
eng_preprocessing(five_shot_fake)
five_shot_fake.head()

,English version,Label,News,Author,Date,URL,only_alphanumeric,imp_text,tokenized_text,stemmed_text
0,Says Daniel Radcliffe has the coronavirus.,entertainment,டேனியல் ராட்கிளிஃப் கொரோனா வைரஸால் பாதிக்கப்பட...,Tweets,"stated on March 10, 2020 in in a tweet:",/factchecks/2020/mar/11/tweets/no-daniel-radcl...,says daniel radcliffe has the coronavirus,says daniel radcliffe coronavirus,"[says, daniel, radcliffe, coronavirus]","[say, daniel, radcliff, coronaviru]"
1,"Says Kobe Bryant's wife ""recorded her suicide ...",entertainment,"கோபி பிரையன்ட்டின் மனைவி ""இன்று காலை மனச்சோர்வ...",Bloggers,"stated on February 3, 2020 in a blog post:",/factchecks/2020/feb/03/blog-posting/headline-...,says kobe bryants wife recorded her suicide af...,says kobe bryants wife recorded suicide depres...,"[says, kobe, bryants, wife, recorded, suicide,...","[say, kobe, bryant, wife, record, suicid, depr..."
2,Says Bill Cosby was framed for sexual assault ...,entertainment,பில் காஸ்பி என்பிசியை வாங்குவதைத் தடுப்பதற்காக...,Facebook posts,"stated on June 14, 2017 in a Facebook post:",/factchecks/2019/nov/04/facebook-posts/no-bill...,says bill cosby was framed for sexual assault ...,says bill cosby framed sexual assault illumina...,"[says, bill, cosby, framed, sexual, assault, i...","[say, bill, cosbi, frame, sexual, assault, ill..."
3,Says a picture he posted on Facebook of a man ...,entertainment,தலை துண்டிக்கப்பட்ட ஐந்து மனிதத் தலைகளுக்கு மு...,Ted Nugent,"stated on June 12, 2014 in a Facebook post:",/factchecks/2014/jun/16/ted-nugent/nugent-ties...,says a picture he posted on facebook of a man ...,says picture posted facebook man posing front ...,"[says, picture, posted, facebook, man, posing,...","[say, pictur, post, facebook, man, pose, front..."
4,"""Statistics show one male cat can father 420,0...",entertainment,"""ஒரு ஆண் பூனை ஐந்து ஆண்டுகளில் 420,000 பூனைக்க...",Central Falls Police Department,"stated on January 28, 2013 in a news release:",/factchecks/2013/feb/02/central-falls-police-d...,statistics show one male cat can father kitte...,statistics show one male cat father kittens fi...,"[statistics, show, one, male, cat, father, kit...","[statist, show, one, male, cat, father, kitten..."


In [25]:
def tamil_preprocessing(dataframe):
  #Remove punctuation symbols
  dataframe['non_punc'] = dataframe['News'].apply(lambda x: re.sub(r'[\'\"\.?!‘]+', '', x.lower()))

  #Tokenize the sentences
  dataframe['tamil_tokenized_text'] = dataframe['non_punc'].apply(lambda x: tokenize(x))

In [26]:
tamil_preprocessing(one_shot_real)
one_shot_real.head()

,English version,Label,News,Author,Date,URL,only_alphanumeric,imp_text,tokenized_text,stemmed_text,non_punc,tamil_tokenized_text
0,Actor Jayam Ravi has Corona,['entertainment'],நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...,actor jayam ravi has corona,actor jayam ravi corona,"[actor, jayam, ravi, corona]","[actor, jayam, ravi, corona]",நடிகர் ஜெயம் ரவிக்கு,"[நடிகர், ஜெயம், ரவிக்கு]"
1,A football god about the 'hands of God',['misc'],‘கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,ஆதி,01-Dec-20,https://www.hindutamil.in/news/supplements/ila...,a football god about the hands of god,football god hands god,"[football, god, hands, god]","[footbal, god, hand, god]",கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,"[கடவுளின், கை’களைப், பற்றிய, கால்பந்துக், கடவுள்]"
2,Cameras mandatory in school buses: Tamil Nadu ...,['politics'],பள்ளி பேருந்துகளில் கேமரா கட்டாயம்: தமிழக அரச...,செய்திப்பிரிவு,21-Oct-22,https://www.hindutamil.in/news/tamilnadu/88576...,cameras mandatory in school buses tamil nadu g...,cameras mandatory school buses tamil nadu gove...,"[cameras, mandatory, school, buses, tamil, nad...","[camera, mandatori, school, buse, tamil, nadu,...",பள்ளி பேருந்துகளில் கேமரா கட்டாயம்: தமிழக அரச...,"[பள்ளி, பேருந்துகளில், கேமரா, கட்டாயம்:, தமிழக..."
3,36th National Games | 4 gold medals for Tamil ...,['sport'],36-வது தேசிய விளையாட்டு | ஒரே நாளில் தமிழகத்த...,செய்திப்பிரிவு,02-Oct-22,https://www.hindutamil.in/news/sports/877427-3...,th national games gold medals for tamil nadu...,th national games gold medals tamil nadu one day,"[th, national, games, gold, medals, tamil, nad...","[th, nation, game, gold, medal, tamil, nadu, o...",36-வது தேசிய விளையாட்டு | ஒரே நாளில் தமிழகத்த...,"[36-வது, தேசிய, விளையாட்டு, |, ஒரே, நாளில், தம..."
4,Introducing NoiseFit Core 2 Smartwatch: Amazin...,['tech'],நாய்ஸ்ஃபிட் கோர் 2 ஸ்மார்ட்வாட்ச் அறிமுகம்: 5...,செய்திப்பிரிவு,22-Aug-22,https://www.hindutamil.in/news/technology/8467...,introducing noisefit core smartwatch amazing ...,introducing noisefit core smartwatch amazing f...,"[introducing, noisefit, core, smartwatch, amaz...","[introduc, noisefit, core, smartwatch, amaz, f...",நாய்ஸ்ஃபிட் கோர் 2 ஸ்மார்ட்வாட்ச் அறிமுகம்: 5...,"[நாய்ஸ்ஃபிட், கோர், 2, ஸ்மார்ட்வாட்ச், அறிமுகம..."


In [27]:
tamil_preprocessing(one_shot_fake)
one_shot_fake.head()

,English version,Label,News,Author,Date,URL,only_alphanumeric,imp_text,tokenized_text,stemmed_text,non_punc,tamil_tokenized_text
0,Says Daniel Radcliffe has the coronavirus.,entertainment,டேனியல் ராட்கிளிஃப் கொரோனா வைரஸால் பாதிக்கப்பட...,Tweets,"stated on March 10, 2020 in in a tweet:",/factchecks/2020/mar/11/tweets/no-daniel-radcl...,says daniel radcliffe has the coronavirus,says daniel radcliffe coronavirus,"[says, daniel, radcliffe, coronavirus]","[say, daniel, radcliff, coronaviru]",டேனியல் ராட்கிளிஃப் கொரோனா வைரஸால் பாதிக்கப்பட...,"[டேனியல், ராட்கிளிஃப், கொரோனா, வைரஸால், பாதிக்..."
1,"Says ""there is no"" COVID-19 virus.",miscellaneous,"COVID-19 வைரஸ் ""இல்லை"" என்கிறார்.",Facebook posts,"stated on March 19, 2020 in a post:",/factchecks/2020/mar/19/facebook-posts/faceboo...,says there is no covid virus,says covid virus,"[says, covid, virus]","[say, covid, viru]",covid-19 வைரஸ் இல்லை என்கிறார்,"[covid-19, வைரஸ், இல்லை, என்கிறார்]"
2,"Says President Donald Trump said, ""People are ...",Politics,"அமெரிக்க அதிபர் டொனால்ட் டிரம்ப், “இதுவரை இறக்...",Facebook posts,"stated on March 18, 2020 in a Facebook post:",/factchecks/2020/mar/19/facebook-posts/no-trum...,says president donald trump said people are dy...,says president donald trump said people dying ...,"[says, president, donald, trump, said, people,...","[say, presid, donald, trump, said, peopl, die,...","அமெரிக்க அதிபர் டொனால்ட் டிரம்ப், “இதுவரை இறக்...","[அமெரிக்க, அதிபர், டொனால்ட், டிரம்ப்,, “இதுவரை..."
3,Kieron Pollard is good: Viral 'Kieron Pollard ...,sport,Kieron Pollard நல்லாதான் இருக்காரு: Viral ஆன ‘...,undefined,"Jan 29, 2021, 07:56 PM IST",https://zeenews.india.com/tamil/social/kieron-...,kieron pollard is good viral kieron pollard de...,kieron pollard good viral kieron pollard death...,"[kieron, pollard, good, viral, kieron, pollard...","[kieron, pollard, good, viral, kieron, pollard...",kieron pollard நல்லாதான் இருக்காரு: viral ஆன k...,"[kieron, pollard, நல்லாதான், இருக்காரு:, viral..."
4,PIB fact check: Link between 5G radiation and ...,tech,PIB fact check: 5 ஜி கதிர்வீச்சுக்கும் COVID-1...,undefined,"May 8, 2021, 11:20 PM IST",https://zeenews.india.com/tamil/social/pib-fac...,pib fact check link between g radiation and se...,pib fact check link g radiation second wave co...,"[pib, fact, check, link, g, radiation, second,...","[pib, fact, check, link, g, radiat, second, wa...",pib fact check: 5 ஜி கதிர்வீச்சுக்கும் covid-1...,"[pib, fact, check:, 5, ஜி, கதிர்வீச்சுக்கும், ..."


In [28]:
tamil_preprocessing(three_shot_real)
three_shot_real.head()

,English version,Label,News,Author,Date,URL,only_alphanumeric,imp_text,tokenized_text,stemmed_text,non_punc,tamil_tokenized_text
0,Actor Jayam Ravi has Corona,['entertainment'],நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...,actor jayam ravi has corona,actor jayam ravi corona,"[actor, jayam, ravi, corona]","[actor, jayam, ravi, corona]",நடிகர் ஜெயம் ரவிக்கு,"[நடிகர், ஜெயம், ரவிக்கு]"
1,Pooja Hegde regrets the film's failure,['entertainment'],படங்கள் தோல்வியால் வருந்தும் பூஜா ஹெக்டே,தினத்தந்தி,22-10-2022 02:50,https://www.dailythanthi.com/Cinema/CinemaNews...,pooja hegde regrets the films failure,pooja hegde regrets films failure,"[pooja, hegde, regrets, films, failure]","[pooja, hegd, regret, film, failur]",படங்கள் தோல்வியால் வருந்தும் பூஜா ஹெக்டே,"[படங்கள், தோல்வியால், வருந்தும், பூஜா, ஹெக்டே]"
2,Images for Diwali,['entertainment'],தீபாவளிக்கு வரும் படங்கள்,தினத்தந்தி,17-10-2022 10:20,https://www.dailythanthi.com/Cinema/CinemaNews...,images for diwali,images diwali,"[images, diwali]","[imag, diwali]",தீபாவளிக்கு வரும் படங்கள்,"[தீபாவளிக்கு, வரும், படங்கள்]"
3,A football god about the 'hands of God',['misc'],‘கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,ஆதி,01-Dec-20,https://www.hindutamil.in/news/supplements/ila...,a football god about the hands of god,football god hands god,"[football, god, hands, god]","[footbal, god, hand, god]",கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,"[கடவுளின், கை’களைப், பற்றிய, கால்பந்துக், கடவுள்]"
4,200 km on two jugs of water. Car trip!,['misc'],இரண்டு குடம் தண்ணீரில் 200 கி.மீ. கார் பயணம்!,அண்டன் பிரகாஷ்,16-Nov-21,https://www.hindutamil.in/news/supplements/ila...,km on two jugs of water car trip,km two jugs water car trip,"[km, two, jugs, water, car, trip]","[km, two, jug, water, car, trip]",இரண்டு குடம் தண்ணீரில் 200 கிமீ கார் பயணம்,"[இரண்டு, குடம், தண்ணீரில், 200, கிமீ, கார், பய..."


In [29]:
tamil_preprocessing(three_shot_fake)
three_shot_fake.head()

,English version,Label,News,Author,Date,URL,only_alphanumeric,imp_text,tokenized_text,stemmed_text,non_punc,tamil_tokenized_text
0,Says Daniel Radcliffe has the coronavirus.,entertainment,டேனியல் ராட்கிளிஃப் கொரோனா வைரஸால் பாதிக்கப்பட...,Tweets,"stated on March 10, 2020 in in a tweet:",/factchecks/2020/mar/11/tweets/no-daniel-radcl...,says daniel radcliffe has the coronavirus,says daniel radcliffe coronavirus,"[says, daniel, radcliffe, coronavirus]","[say, daniel, radcliff, coronaviru]",டேனியல் ராட்கிளிஃப் கொரோனா வைரஸால் பாதிக்கப்பட...,"[டேனியல், ராட்கிளிஃப், கொரோனா, வைரஸால், பாதிக்..."
1,"Says Kobe Bryant's wife ""recorded her suicide ...",entertainment,"கோபி பிரையன்ட்டின் மனைவி ""இன்று காலை மனச்சோர்வ...",Bloggers,"stated on February 3, 2020 in a blog post:",/factchecks/2020/feb/03/blog-posting/headline-...,says kobe bryants wife recorded her suicide af...,says kobe bryants wife recorded suicide depres...,"[says, kobe, bryants, wife, recorded, suicide,...","[say, kobe, bryant, wife, record, suicid, depr...",கோபி பிரையன்ட்டின் மனைவி இன்று காலை மனச்சோர்வு...,"[கோபி, பிரையன்ட்டின், மனைவி, இன்று, காலை, மனச்..."
2,Says Bill Cosby was framed for sexual assault ...,entertainment,பில் காஸ்பி என்பிசியை வாங்குவதைத் தடுப்பதற்காக...,Facebook posts,"stated on June 14, 2017 in a Facebook post:",/factchecks/2019/nov/04/facebook-posts/no-bill...,says bill cosby was framed for sexual assault ...,says bill cosby framed sexual assault illumina...,"[says, bill, cosby, framed, sexual, assault, i...","[say, bill, cosbi, frame, sexual, assault, ill...",பில் காஸ்பி என்பிசியை வாங்குவதைத் தடுப்பதற்காக...,"[பில், காஸ்பி, என்பிசியை, வாங்குவதைத், தடுப்பத..."
3,"Says ""there is no"" COVID-19 virus.",miscellaneous,"COVID-19 வைரஸ் ""இல்லை"" என்கிறார்.",Facebook posts,"stated on March 19, 2020 in a post:",/factchecks/2020/mar/19/facebook-posts/faceboo...,says there is no covid virus,says covid virus,"[says, covid, virus]","[say, covid, viru]",covid-19 வைரஸ் இல்லை என்கிறார்,"[covid-19, வைரஸ், இல்லை, என்கிறார்]"
4,Says a mall Santa beat up a child molester.,miscellaneous,ஒரு மால் சாண்டா ஒரு குழந்தை துஷ்பிரயோகம் செய்ப...,Bloggers,"stated on November 7, 2019 in a Facebook post:",/factchecks/2019/nov/26/blog-posting/dont-fall...,says a mall santa beat up a child molester,says mall santa beat child molester,"[says, mall, santa, beat, child, molester]","[say, mall, santa, beat, child, molest]",ஒரு மால் சாண்டா ஒரு குழந்தை துஷ்பிரயோகம் செய்ப...,"[ஒரு, மால், சாண்டா, ஒரு, குழந்தை, துஷ்பிரயோகம்..."


In [30]:
tamil_preprocessing(five_shot_real)
five_shot_real.head()

,English version,Label,News,Author,Date,URL,only_alphanumeric,imp_text,tokenized_text,stemmed_text,non_punc,tamil_tokenized_text
0,Actor Jayam Ravi has Corona,['entertainment'],நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...,actor jayam ravi has corona,actor jayam ravi corona,"[actor, jayam, ravi, corona]","[actor, jayam, ravi, corona]",நடிகர் ஜெயம் ரவிக்கு,"[நடிகர், ஜெயம், ரவிக்கு]"
1,Pooja Hegde regrets the film's failure,['entertainment'],படங்கள் தோல்வியால் வருந்தும் பூஜா ஹெக்டே,தினத்தந்தி,22-10-2022 02:50,https://www.dailythanthi.com/Cinema/CinemaNews...,pooja hegde regrets the films failure,pooja hegde regrets films failure,"[pooja, hegde, regrets, films, failure]","[pooja, hegd, regret, film, failur]",படங்கள் தோல்வியால் வருந்தும் பூஜா ஹெக்டே,"[படங்கள், தோல்வியால், வருந்தும், பூஜா, ஹெக்டே]"
2,Images for Diwali,['entertainment'],தீபாவளிக்கு வரும் படங்கள்,தினத்தந்தி,17-10-2022 10:20,https://www.dailythanthi.com/Cinema/CinemaNews...,images for diwali,images diwali,"[images, diwali]","[imag, diwali]",தீபாவளிக்கு வரும் படங்கள்,"[தீபாவளிக்கு, வரும், படங்கள்]"
3,Simbu debuts in Bollywood,['entertainment'],பாலிவுட்டில் அறிமுகமாகிறார் சிம்பு,தினத்தந்தி,16-10-2022 03:49,https://www.dailythanthi.com/Cinema/CinemaNews...,simbu debuts in bollywood,simbu debuts bollywood,"[simbu, debuts, bollywood]","[simbu, debut, bollywood]",பாலிவுட்டில் அறிமுகமாகிறார் சிம்பு,"[பாலிவுட்டில், அறிமுகமாகிறார், சிம்பு]"
4,3rd song release from 'Prince',['entertainment'],'பிரின்ஸ்' படத்தின் 3வது பாடல் வெளியீடு,தினத்தந்தி,14-10-2022 16:31,https://www.dailythanthi.com/Cinema/CinemaNews...,rd song release from prince,rd song release prince,"[rd, song, release, prince]","[rd, song, releas, princ]",பிரின்ஸ் படத்தின் 3வது பாடல் வெளியீடு,"[பிரின்ஸ், படத்தின், 3வது, பாடல், வெளியீடு]"


In [31]:
tamil_preprocessing(five_shot_fake)
five_shot_fake.head()

,English version,Label,News,Author,Date,URL,only_alphanumeric,imp_text,tokenized_text,stemmed_text,non_punc,tamil_tokenized_text
0,Says Daniel Radcliffe has the coronavirus.,entertainment,டேனியல் ராட்கிளிஃப் கொரோனா வைரஸால் பாதிக்கப்பட...,Tweets,"stated on March 10, 2020 in in a tweet:",/factchecks/2020/mar/11/tweets/no-daniel-radcl...,says daniel radcliffe has the coronavirus,says daniel radcliffe coronavirus,"[says, daniel, radcliffe, coronavirus]","[say, daniel, radcliff, coronaviru]",டேனியல் ராட்கிளிஃப் கொரோனா வைரஸால் பாதிக்கப்பட...,"[டேனியல், ராட்கிளிஃப், கொரோனா, வைரஸால், பாதிக்..."
1,"Says Kobe Bryant's wife ""recorded her suicide ...",entertainment,"கோபி பிரையன்ட்டின் மனைவி ""இன்று காலை மனச்சோர்வ...",Bloggers,"stated on February 3, 2020 in a blog post:",/factchecks/2020/feb/03/blog-posting/headline-...,says kobe bryants wife recorded her suicide af...,says kobe bryants wife recorded suicide depres...,"[says, kobe, bryants, wife, recorded, suicide,...","[say, kobe, bryant, wife, record, suicid, depr...",கோபி பிரையன்ட்டின் மனைவி இன்று காலை மனச்சோர்வு...,"[கோபி, பிரையன்ட்டின், மனைவி, இன்று, காலை, மனச்..."
2,Says Bill Cosby was framed for sexual assault ...,entertainment,பில் காஸ்பி என்பிசியை வாங்குவதைத் தடுப்பதற்காக...,Facebook posts,"stated on June 14, 2017 in a Facebook post:",/factchecks/2019/nov/04/facebook-posts/no-bill...,says bill cosby was framed for sexual assault ...,says bill cosby framed sexual assault illumina...,"[says, bill, cosby, framed, sexual, assault, i...","[say, bill, cosbi, frame, sexual, assault, ill...",பில் காஸ்பி என்பிசியை வாங்குவதைத் தடுப்பதற்காக...,"[பில், காஸ்பி, என்பிசியை, வாங்குவதைத், தடுப்பத..."
3,Says a picture he posted on Facebook of a man ...,entertainment,தலை துண்டிக்கப்பட்ட ஐந்து மனிதத் தலைகளுக்கு மு...,Ted Nugent,"stated on June 12, 2014 in a Facebook post:",/factchecks/2014/jun/16/ted-nugent/nugent-ties...,says a picture he posted on facebook of a man ...,says picture posted facebook man posing front ...,"[says, picture, posted, facebook, man, posing,...","[say, pictur, post, facebook, man, pose, front...",தலை துண்டிக்கப்பட்ட ஐந்து மனிதத் தலைகளுக்கு மு...,"[தலை, துண்டிக்கப்பட்ட, ஐந்து, மனிதத், தலைகளுக்..."
4,"""Statistics show one male cat can father 420,0...",entertainment,"""ஒரு ஆண் பூனை ஐந்து ஆண்டுகளில் 420,000 பூனைக்க...",Central Falls Police Department,"stated on January 28, 2013 in a news release:",/factchecks/2013/feb/02/central-falls-police-d...,statistics show one male cat can father kitte...,statistics show one male cat father kittens fi...,"[statistics, show, one, male, cat, father, kit...","[statist, show, one, male, cat, father, kitten...","ஒரு ஆண் பூனை ஐந்து ஆண்டுகளில் 420,000 பூனைக்கு...","[ஒரு, ஆண், பூனை, ஐந்து, ஆண்டுகளில், 420,000, ப..."


In [32]:
#Apply TF-IDF
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()
    train = tfidf_vectorizer.fit_transform(data)
    return train, tfidf_vectorizer